In [ ]:
pip install datasets trl

In [ ]:
!pip install -U bitsandbytes

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig
import pandas as pd
from datasets import load_dataset

In [ ]:
dataset = load_dataset("choeyunbeom/novel_title", split = 'train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

Dataset({
    features: ['cleaned_sub_title', 'cleaned_content'],
    num_rows: 20262
})

In [ ]:
def generate_prompts(example):
    prompt_list = []
    for i in range(len(example['cleaned_content'])):
        prompt_list.append(
        f"""<|begin_of_text|>
        <|start_header_id|>user<|end_header_id|>다음 글의 제목을 지어주세요:
        {example['cleaned_content'][i]}<|eot_id|><|start_header_id|>assistant<|end_header_id|>
        {example['cleaned_sub_title'][i]}<|eot_id|>"""
                )
    return prompt_list

In [ ]:
train_test_split = dataset.train_test_split(test_size=0.2, seed=42)

train_dataset = train_test_split['train']
test_dataset = train_test_split['test']

In [ ]:
# 베이스 모델 정의,
base_model = 'choeyunbeom/llama3_KM'

if torch.cuda.get_device_capability()[0] >= 8: # Attention 메커니즘 선택:
    !pip install -qqq flash-attn
    attn_implementation = "flash_attention_2"
    torch_dtype = torch.bfloat16
else:
    attn_implementation = "eager"
    torch_dtype = torch.float16

In [ ]:
# QLoRA config
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,                       # 모델 가중치 4비트로 로드
    bnb_4bit_quant_type="nf4",               # 양자화 유형 : "nf4"
    bnb_4bit_compute_dtype=torch_dtype,      # 양자화를 위한 컴퓨팅 타입
    bnb_4bit_use_double_quant=False,         # 이중 양자화 x
)

In [ ]:
# 사전학습된 모델 load
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config, # 정의한 양자화 설정
    device_map= {"":0}
)
model.config.use_cache = False        # 모델의 캐시 기능 비활성화
model.config.pretraining_tp = 1       # 모델의 텐서 병렬화

model-00001-of-00002.safetensors:   8%|8         | 398M/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# 토크나이저 load
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    trust_remote_code = True,   # 원격 코드의 신뢰 여부 설정.
    # add_special_tokens=True
)
tokenizer.pad_token = tokenizer.eos_token  # 문장 끝에 eos_token
tokenizer.padding_side = 'right'           #패딩 토큰을 오른쪽에 추가

In [ ]:
# LoRA
peft_params = LoraConfig(
    lora_alpha = 16,                         # 스케일링 계수, 값이 클수록 학습 속도 빨라짐
    lora_dropout = 0.1,                      # 드롭아웃
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj"],   # 최적화하려는 모델의 모듈을 지정
    r= 32,                                   # LoRA 어댑터 행렬의 Rank, 랭크가 높을수록 모델의 표현 능력 향상
    bias = 'none',                           # 어댑터 행렬에 대한 편향 추가 여부
    task_type = 'CAUSAL_LM'                  # LoRA가 적용 될 작업 유형
)

In [ ]:
# 모델 학습 과정에 필요한 설정
training_params = TrainingArguments(
    output_dir = "./results",           # 학습 결과 저장 디렉토리, 모델 가중치, 로그, 체크포인트
    num_train_epochs = 2,               # 전체 학습 데이터 셋 반복횟수
    per_device_train_batch_size = 2,    # 배치 크기
    gradient_accumulation_steps = 1,    # 여러 배치에서 계산된 기울기를 누적하여 실제 가중치 업데이트 수행 빈도
    optim = 'paged_adamw_32bit',        # 옵티마이저
    save_steps = 100,                    # 스텝마다 모델 기록, 저장
    logging_steps = 100,                 # 스텝마다 로그 기록, 저장
    learning_rate = 2e-4,               # 학습률
    weight_decay = 0.001,               # 가중치 감소 계수, 정규화 기법
    fp16 = True,                        # 16비트 부동소수점 정밀도 사용 여부
    bf16 = False,                       # Brain Floating16 연산 사용여부
    max_grad_norm = 0.3,                # 기울기의 최대 Norm, 기울기 폭발 방지
    max_steps = -1,                    # 최대 학습 스텝 수, -1 -> num_train_epochs 동안
    warmup_ratio = 0.03,                # 학습율 워밍업에 사용할 스텝 비율. 학습률 점진적으로 증가
    group_by_length = True,             # 입력 시퀀스의 길이에 따라 배치 그룹화 여부
    lr_scheduler_type = 'constant',       # 학습률 스케쥴러 유형
    report_to = 'tensorboard'           # 학습 로그 기록할 툴
)

In [ ]:
# training_arguments = SFTConfig()
trainer = SFTTrainer(
    model = model,
    train_dataset = train_dataset,
    formatting_func= generate_prompts,
    eval_dataset= test_dataset,
    dataset_text_field="cleaned_content",
    peft_config = peft_params,
    max_seq_length=2048,
    tokenizer = tokenizer,
    args = training_params,
    packing= False  # 동일한 입력 시퀀스에 여러 개의 짧은 예제를 넣어 효율성을 높일 수 있음
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/16209 [00:00<?, ? examples/s]

Map:   0%|          | 0/4053 [00:00<?, ? examples/s]

In [ ]:
torch.cuda.empty_cache()

In [ ]:
trainer.train()

Step,Training Loss
100,2.638900
200,2.575900
300,2.568700
400,2.573700
500,2.532000
600,2.513500
700,2.534800
800,2.512600
900,2.506600
1000,2.509600


TrainOutput(global_step=16210, training_loss=2.4259078157014278, metrics={'train_runtime': 11835.0986, 'train_samples_per_second': 2.739, 'train_steps_per_second': 1.37, 'total_flos': 1.1134478526290043e+18, 'train_loss': 2.4259078157014278, 'epoch': 2.0})

In [ ]:
trainer.model.save_pretrained("/content/drive/MyDrive/llama3")

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
   'choeyunbeom/llama3_KM',
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16
)
model2 = PeftModel.from_pretrained(base_model, "/content/drive/MyDrive/llama3") # LoRA 가중치를 가져와 기본 모델에 통합

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
model2 = model2.merge_and_unload()

In [ ]:
# 사전 훈련된 토크나이저를 다시 로드
tokenizer = AutoTokenizer.from_pretrained('choeyunbeom/llama3_KM', trust_remote_code=True)

# 토크나이저의 패딩 토큰을 종료 토큰(end-of-sentence token)과 동일하게 설정
tokenizer.pad_token = tokenizer.eos_token

# 패딩을 시퀀스의 오른쪽에 적용
tokenizer.padding_side = "right"

In [ ]:
model2.push_to_hub("choeyunbeom/llama3_KM")
tokenizer.push_to_hub("choeyunbeom/llama3_KM")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/choeyunbeom/llama3_KM/commit/85e28d73360ed8e91093ee550223c4d97a919255', commit_message='Upload tokenizer', commit_description='', oid='85e28d73360ed8e91093ee550223c4d97a919255', pr_url=None, repo_url=RepoUrl('https://huggingface.co/choeyunbeom/llama3_KM', endpoint='https://huggingface.co', repo_type='model', repo_id='choeyunbeom/llama3_KM'), pr_revision=None, pr_num=None)

In [ ]:
torch.cuda.empty_cache()